import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [2]:
spark_session = (SparkSession.builder
    .appName("Local PySpark Application")
    .master("local[*]")
    .getOrCreate())
print(spark_session.version)

3.5.3


read csv with inferschema

In [3]:
%%time
df_1 = spark_session.read.csv("data/ds_salaries.csv", header=True, inferSchema=True)

CPU times: total: 15.6 ms
Wall time: 5.98 s


read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [4]:
%%time
df_2 = spark_session.read.csv("data/ds_salaries.csv", header=True, inferSchema=True)

CPU times: total: 0 ns
Wall time: 230 ms


write schema of scv on screen

In [5]:
df_1.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [3]:
ds_salaries_schema = StructType([
    StructField("id", IntegerType(), nullable=True),
    StructField("work_year", IntegerType(), nullable=True),
    StructField("experience_level", StringType(), nullable=True),
    StructField("employment_type", StringType(), nullable=True),
    StructField("job_title", StringType(), nullable=True),
    StructField("salary", IntegerType(), nullable=True),
    StructField("salary_currency", StringType(), nullable=True),
    StructField("salary_in_usd", IntegerType(), nullable=True),
    StructField("employee_residence", StringType(), nullable=True),
    StructField("remore_ratio", IntegerType(), nullable=True),
    StructField("company_location", StringType(), nullable=True),
    StructField("company_size", StringType(), nullable=True),
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [4]:
%%time
df3 = spark_session.read.csv("data/ds_salaries.csv", schema=ds_salaries_schema)

CPU times: total: 0 ns
Wall time: 2.32 s


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [5]:
df3.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remore_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [6]:
df3.show()

+----+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remore_ratio|company_location|company_size|
+----+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|NULL|     NULL|experience_level|employment_type|           job_title|    NULL|salary_currency|         NULL|employee_residence|        NULL|company_location|company_size|
|   0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|   1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0

print data in dataframe using display(df.toPandas())

In [8]:
display(df3.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remore_ratio,company_location,company_size
0,NaN,NaN,experience_level,employment_type,job_title,NaN,salary_currency,NaN,employee_residence,NaN,company_location,company_size
1,0.0,2020.0,MI,FT,Data Scientist,70000.0,EUR,79833.0,DE,0.0,DE,L
2,1.0,2020.0,SE,FT,Machine Learning Scientist,260000.0,USD,260000.0,JP,0.0,JP,S
3,2.0,2020.0,SE,FT,Big Data Engineer,85000.0,GBP,109024.0,GB,50.0,GB,M
4,3.0,2020.0,MI,FT,Product Data Analyst,20000.0,USD,20000.0,HN,0.0,HN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
603,602.0,2022.0,SE,FT,Data Engineer,154000.0,USD,154000.0,US,100.0,US,M
604,603.0,2022.0,SE,FT,Data Engineer,126000.0,USD,126000.0,US,100.0,US,M
605,604.0,2022.0,SE,FT,Data Analyst,129000.0,USD,129000.0,US,0.0,US,M
606,605.0,2022.0,SE,FT,Data Analyst,150000.0,USD,150000.0,US,100.0,US,M


create df_job_title that consists from all job_titles without duplicates

In [14]:
df_3_stable = df3.exceptAll(df3.limit(1))
df_job_title = df_3_stable.select("job_title").distinct()

print all rows from df_job_titles without truncating jobs

In [15]:
df_job_title.show(df_job_title.count(), truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
|Research S

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [31]:
df_analytic = df_3_stable.groupBy('job_title').agg(
    max('salary_in_usd').alias('max_salary'),
    min('salary_in_usd').alias('min_salary'),
    avg('salary_in_usd').alias('avg_salary')
)

print all rows from df_analytic without trancating jobs

In [32]:
df_analytic.show(df_analytic.count(), truncate=False)

+----------------------------------------+----------+----------+------------------+
|job_title                               |max_salary|min_salary|avg_salary        |
+----------------------------------------+----------+----------+------------------+
|3D Computer Vision Researcher           |5409      |5409      |5409.0            |
|Lead Data Engineer                      |276000    |56000     |139724.5          |
|Head of Machine Learning                |79039     |79039     |79039.0           |
|Data Specialist                         |165000    |165000    |165000.0          |
|Data Analytics Lead                     |405000    |405000    |405000.0          |
|Machine Learning Scientist              |260000    |12000     |158412.5          |
|Lead Data Analyst                       |170000    |19609     |92203.0           |
|Data Engineering Manager                |174000    |59303     |123227.2          |
|Staff Data Scientist                    |105000    |105000    |105000.0    

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [33]:
window_spec = Window.orderBy(col('avg_salary').desc())
df_analytic = df_analytic.withColumn('row_id', row_number().over(window_spec))

print all data from df_analytic

In [34]:
df_analytic.show(df_analytic.count(), truncate=False)

+----------------------------------------+----------+----------+------------------+------+
|job_title                               |max_salary|min_salary|avg_salary        |row_id|
+----------------------------------------+----------+----------+------------------+------+
|Data Analytics Lead                     |405000    |405000    |405000.0          |1     |
|Principal Data Engineer                 |600000    |185000    |328333.3333333333 |2     |
|Financial Data Analyst                  |450000    |100000    |275000.0          |3     |
|Principal Data Scientist                |416000    |148261    |215242.42857142858|4     |
|Director of Data Science                |325000    |130026    |195074.0          |5     |
|Data Architect                          |266400    |90700     |177873.9090909091 |6     |
|Applied Data Scientist                  |380000    |54238     |175655.0          |7     |
|Analytics Engineer                      |205300    |135000    |175000.0          |8     |

it isn't beautifull, so we need to put now row_id on first place in df_analytic

print df_analytic now

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

print here df_exp_lvl

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

print df_best

drop duplicates if exist by experience_level

print df_best

create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

print df_new_best

write df_new_best like 1.csv and load then it to df_final

print df_final

filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

print df_final

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system